In [10]:
from bs4 import BeautifulSoup
import json
import pandas as pd
from pandas import DataFrame
import numpy as np
import re
import requests
import xml.etree.ElementTree as et

In [11]:
url = 'https://www.thestorefront.com/sitemap.xml'
r = requests.get(url)
xml = r.text

In [12]:
soup = BeautifulSoup(xml)
locs = soup.find_all('loc')

In [ ]:
raw_pages = {}
n = 0

for i, v in enumerate(locs):
    pattern = re.compile(r'/united-states/new-york/')
    to_search = v.text
    if pattern.search(to_search):
        r = requests.get(to_search)
        attrib = BeautifulSoup(r.text)
        raw_pages[to_search] = attrib.find_all('div', {'class': 'information'})
        raw_pages[to_search].extend(attrib.find_all('div', {'class': 'listing-information'}))

    if n > 0:
        if len(list(raw_pages.keys())) > n: break

In [ ]:
attributes = {
    'Pricing': r'class="pricing-value" content="[\d]+"',
    # 'Price Per Day': r'<span class="pricing-value">[\d\$\,\.]+</span>',
    'Minimum': r'Minimum Rental.*[\w\d ]+</div>',
    'Maximum': r'Maximum Rental.*[\w\d ]+</div>',
    'Space Type': r'Space Type.*id="space-types-link">[\w ]+</a>',
    'FloorAccess': r'Floor/Access.*[\w ]+</div>',
    'Rental Capacity': r'Rental Capacity.*[\w ]+</div>',
    'Size': r'Size.*[\d\w, ]+</div>',
    'Ref': r'Ref Number.*[\d]+</div>'
}

In [ ]:
parsed = {}
for page in raw_pages:
    parsed[page] = {}
    for prop in raw_pages[page]:
        for attrib in attributes:
            pattern = re.compile(attributes[attrib])
            matches = pattern.search(str(prop))
            if matches:
                parsed[page][attrib] = matches.group(0)

In [ ]:
target_values = {
    # 'Price Per Day': r'[\d\$\,\.]+',
    'Pricing': r'[\d]+',
    'Minimum': r'[\w\d ]+',
    'Maximum': r'[\w\d ]+',
    'Space Type': r'[\w ]+',
    'FloorAccess': r'[\w ]+',
    'Rental Capacity': r'[\w ]+',
    'Size': r'[\d\w, ]+',
    'Ref': r'[\d]+'
}

In [ ]:
cleaned = {}
for parse in parsed:
    cleaned[parse] = {}
    for prop in parsed[parse]:
        for attrib in target_values:
            if not attrib == 'Pricing':
                pat = f'>{target_values[attrib]}<'
            else:
                pat = target_values[attrib]
            pattern = re.compile(pat)
            matches = pattern.search(parsed[parse][prop])
            if matches:
                cleaned[parse][prop] = re.sub('<|>', '', matches.group(0))

In [ ]:
df = DataFrame(cleaned.values())
df['URL'] = list(cleaned.keys())
df.to_csv(f'prep_storefront_first_{n}_results.csv', index=False)